In [102]:
import xarray as xr
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pickle
from scipy.ndimage.filters import uniform_filter1d
from matplotlib.gridspec import GridSpec
import geopandas as gpd
import regionmask
from scipy.stats import linregress
import statsmodels.api as sm

from glob import glob

import matplotlib.path as mpath
import cartopy
import cartopy.crs as ccrs
crs_sp = ccrs.SouthPolarStereo(central_longitude=0)
crs_ll = ccrs.PlateCarree()

from cartopy.util import add_cyclic_point

In [105]:
def plot_antarctica(ax, lat):
    coast_line = gpd.read_file('/glade/work/devond/shp_files/Coastline_Antarctica_v2.shp')
    grounding_line = gpd.read_file('/glade/work/devond/shp_files/GroundingLine_Antarctica_v2.shp')
    
    theta = np.linspace(0, 2*np.pi, 100)
    center, radius = [0.5, 0.5], 0.51
    verts = np.vstack([np.sin(theta), np.cos(theta)]).T
    circle = mpath.Path(verts * radius + center)
    
    ax.add_geometries(coast_line['geometry'], crs = ccrs.SouthPolarStereo(central_longitude=0), facecolor='none', edgecolor='black', linewidth = 0.5)
    ax.add_geometries(grounding_line['geometry'], crs = ccrs.SouthPolarStereo(central_longitude=0), facecolor='none', edgecolor='black', linewidth = 0.5)
    ax.set_boundary(circle, transform=ax.transAxes)
    ax.set_extent([-180, 180, -90, lat], ccrs.PlateCarree())
    return

#ross sea = 'Ross Ice Shelf' (69,70), 'Nickerson' (63), 'Sulzberger' (64)
#Amundsen Sea = 'Crosson' (56), 'Dotson' (57), 'Getz' (59), 'Thwaites' (55), 'Abbot' (109), 'Pine_Island' (54), 'Venable' (52), 'Cosgrove' (53)
#Amery Region = 'Amery' (114), 'West' (101)
#DML = 'Jelbart' (124), 'Brunt_Stancomb' (35), 'Riiser-Larsen' (36), 'Ekstrom' (38), 'Atka' (123), 'Prince_Harald' (157), 
    # 'Fimbul' (120), 'Borchgrevink' (118), 'Baudouin' (117), 'Quar' (37), 'Lazarev' (119), 'Nivl' (121)
# pen = 'Stange' (51), 'George_VI' (44), 'LarsenD' (5), 'LarsenC' (7), 'Bach' (45), 'Wilkins' (49), 'LarsenB' (50)
#RF = 'Ronne' (33), 'Filchner' (34)
#victoria = 'Mertz' (90), 'Cook' (89), 'Mariner' (82), 'Drygalski' (72)
#wilkes = 'Shackleton' (99), 'Tracy_Tremenchus' (23), 'Moscow_University' (156), 'Totten' (97), 'Holmes' (155)

def get_shelf_mask(area):

    if area == 'ross':
        mask = shelf_mask_da.where((shelf_mask_da==69)|(shelf_mask_da==70)|
                                  (shelf_mask_da==63)|(shelf_mask_da==64))
    elif area == 'rf':#ronne-filchner
        mask = shelf_mask_da.where((shelf_mask_da==33)|(shelf_mask_da==34))
    elif area == 'pen':#larsen c & d, G6, Bach, wilkins
        mask = shelf_mask_da.where((shelf_mask_da==51)|(shelf_mask_da==44)|(shelf_mask_da==5)|
                                   (shelf_mask_da==7)|(shelf_mask_da==45)|(shelf_mask_da==49)|
                                   (shelf_mask_da==50))
    elif area == 'amery':
        mask = shelf_mask_da.where((shelf_mask_da==114)|(shelf_mask_da==101))
    elif area == 'wilkes': 
        mask = shelf_mask_da.where((shelf_mask_da==99)|(shelf_mask_da==23)|(shelf_mask_da==156)
                                  |(shelf_mask_da==97)|(shelf_mask_da==155))
    elif area == 'a-b':
        mask = shelf_mask_da.where((shelf_mask_da==56)|(shelf_mask_da==57)|(shelf_mask_da==55)|(shelf_mask_da==59)|
                                   (shelf_mask_da==54)|(shelf_mask_da==109)|(shelf_mask_da==52)|(shelf_mask_da==53))

    elif area == 'victoria':
        mask = shelf_mask_da.where((shelf_mask_da==90)|(shelf_mask_da==89)|(shelf_mask_da==82)|(shelf_mask_da==72))
    elif area == 'DML':
        mask = shelf_mask_da.where((shelf_mask_da==124)|(shelf_mask_da==35)|(shelf_mask_da==36)|(shelf_mask_da==38)
                                  |(shelf_mask_da==123)|(shelf_mask_da==157)|(shelf_mask_da==120)
                                  |(shelf_mask_da==118)|(shelf_mask_da==117)|(shelf_mask_da==37)
                                  |(shelf_mask_da==119)|(shelf_mask_da==121))
    elif area == 'all':
        mask = shelf_mask_da.where((shelf_mask_da==69)|(shelf_mask_da==70)|(shelf_mask_da==33)|
                                   (shelf_mask_da==34)|(shelf_mask_da==51)|(shelf_mask_da==44)|
                                   (shelf_mask_da==5)|(shelf_mask_da==7)|(shelf_mask_da==45)|
                                   (shelf_mask_da==49)|(shelf_mask_da==50)|(shelf_mask_da==114)|
                                   (shelf_mask_da==99)|(shelf_mask_da==101)|(shelf_mask_da==23)|
                                   (shelf_mask_da==56)|(shelf_mask_da==57)|(shelf_mask_da==55)|
                                   (shelf_mask_da==59)|(shelf_mask_da==54)|(shelf_mask_da==109)|
                                   (shelf_mask_da==52)|(shelf_mask_da==53)|(shelf_mask_da==90)|
                                   (shelf_mask_da==97)|(shelf_mask_da==156)|(shelf_mask_da==89)|
                                   (shelf_mask_da==63)|(shelf_mask_da==64)|(shelf_mask_da==72)|
                                   (shelf_mask_da==124)|(shelf_mask_da==35)|(shelf_mask_da==36)|
                                   (shelf_mask_da==38)|(shelf_mask_da==123)|(shelf_mask_da==157)|
                                   (shelf_mask_da==120)|(shelf_mask_da==118)|(shelf_mask_da==117)|
                                   (shelf_mask_da==37)|(shelf_mask_da==119)|(shelf_mask_da==121)|
                                   (shelf_mask_da==82)|(shelf_mask_da==155))

        
    mask = mask*el_mask_da.values
        
    mask2 = mask.dropna(dim = 'lon', how='all')
    mask2 = mask2.dropna(dim = 'lat', how='all')
    mask_lat = mask2.lat.values
    mask_lon = mask2.lon.values
    
    return mask, mask_lat, mask_lon

def clip_data_to_mask(area):
    mask, mask_lat, mask_lon= get_shelf_mask(area)

    mask_np = mask.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon))).values
    mask_np[~np.isnan(mask_np)]=1
    
    low_el_mask = el_mask_da.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon))).values
    
    era_area = area_da.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
    shelf_area = (mask_np*era_area*low_el_mask).sum(dim = ['lat','lon'])

    clipped_ssp5 = ssp5_data.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
    ssp5_ts = (clipped_ssp5*mask*era_area*low_el_mask).sum(dim = ['lat','lon'])/shelf_area

    clipped_ssp3 = ssp3_data.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
    ssp3_ts = (clipped_ssp3*mask*era_area*low_el_mask).sum(dim = ['lat','lon'])/shelf_area

    clipped_ssp1 = ssp1_data.sel(lat = slice(np.max(mask_lat), np.min(mask_lat)), lon = slice(np.min(mask_lon), np.max(mask_lon)))
    ssp1_ts = (clipped_ssp1*mask*era_area*low_el_mask).sum(dim = ['lat','lon'])/shelf_area
    
    return ssp1_ts, ssp3_ts, ssp5_ts, mask

In [106]:
el_mask_da = xr.open_dataarray('masks/era/low_elevation.nc')
shelf_mask_da = xr.open_dataarray('masks/era/shelf_mask.nc')
area_da = xr.open_dataarray('masks/era/area.nc')

In [115]:
run = 'deg0C'

ssp1_data = xr.open_dataset(f'netcdf_results/{run}/ssp1_fac.nc')
ssp3_data = xr.open_dataset(f'netcdf_results/{run}/ssp3_fac.nc')
ssp5_data = xr.open_dataset(f'netcdf_results/{run}/ssp5_fac.nc')

In [116]:
#for pen, rf, amery, wilkes, ross, victoria, a-b, DML, all
for r in ['pen', 'rf', 'amery', 'wilkes', 'ross', 'victoria', 'a-b', 'DML', 'all']:
    print(r)
    ssp1_clip, ssp3_clip, ssp5_clip, mask = clip_data_to_mask(r)
    ds = xr.Dataset(
    data_vars=dict(
        ssp1=(["model", "year"], ssp1_clip.fac.values),
        ssp3=(["model", "year"], ssp3_clip.fac.values),
        ssp5=(["model", "year"], ssp5_clip.fac.values),
        ),
        coords=dict(
            model=ssp1_clip.model.values,
            year=ssp1_clip.year.values,
        )
        )
    ds.to_netcdf(f'timeseries/{run}/{r}.nc')

pen
rf
amery
wilkes
ross
victoria
a-b
DML
all
